In [1]:
#dependancies
import pandas as pd
import datetime as dt
import calendar

#Create list of dataframes from csv files
f1 = "JC-202011-citibike-tripdata.csv"
f2 = "JC-202012-citibike-tripdata.csv"
file_list = [f1,f2]
for i in range(1,10):
    file_list.append(f"JC-20210{i}-citibike-tripdata.csv")
for i in range(0,3):
    file_list.append(f"JC-20211{i}-citibike-tripdata.csv") 
df_list = [pd.read_csv(file) for file in file_list]

#Split dataframes into two lists because data formats changed in February 2021
df_list1 = df_list[0:3]
df_list2 = df_list[3:14]

In [2]:
#Harmonize dataframes
for dataf in df_list1:
    #Drop columns that are not in common
    dataf.drop(['tripduration', 'start station id','end station id','bikeid','birth year','gender'], axis = 1,inplace=True)
    #Rename columns to match new format
    dataf.columns = ['started_at','ended_at','start_station_name','start_lat','start_lng','end_station_name','end_lat','end_lng','member_casual']

for dataf in df_list2:
    #Drop columns that are not in common
    dataf.drop(['ride_id', 'rideable_type','start_station_id','end_station_id'], axis = 1,inplace=True)

#Recreate full dataframe list
df_list1.extend(df_list2)

#Create master dataframe from all individual dataframes (1 for each month of data)
df = df_list1[0].append(df_list1[1::],ignore_index=True)
df

,started_at,ended_at,start_station_name,start_lat,start_lng,end_station_name,end_lat,end_lng,member_casual
0,2020-11-01 00:11:32.6250,2020-11-01 00:24:09.5850,Columbus Drive,40.718355,-74.038914,Baldwin at Montgomery,40.723659,-74.064194,Subscriber
1,2020-11-01 00:14:20.4820,2020-11-01 00:24:04.7820,Grand St,40.715178,-74.037683,Brunswick St,40.724176,-74.050656,Customer
2,2020-11-01 00:22:55.8820,2020-11-01 00:29:39.0950,Hamilton Park,40.727596,-74.044247,Montgomery St,40.719420,-74.050990,Subscriber
3,2020-11-01 00:41:37.4270,2020-11-01 00:49:55.7010,Lafayette Park,40.713464,-74.062859,Van Vorst Park,40.718489,-74.047727,Customer
4,2020-11-01 00:53:21.2340,2020-11-01 00:55:52.8060,City Hall,40.717732,-74.043845,Marin Light Rail,40.714584,-74.042817,Subscriber
...,...,...,...,...,...,...,...,...,...
677320,2021-12-16 16:15:22,2021-12-16 16:32:43,Oakland Ave,40.737604,-74.052478,8 St & Washington St,40.745984,-74.028199,member
677321,2021-12-27 23:57:45,2021-12-28 00:00:23,Clinton St & 7 St,40.745420,-74.033320,8 St & Washington St,40.745984,-74.028199,casual
677322,2021-12-21 13:32:14,2021-12-21 13:48:23,Clinton St & 7 St,40.745420,-74.033320,8 St & Washington St,40.745984,-74.028199,member
677323,2021-12-22 12:25:55,2021-12-22 12:34:17,Grand St & 14 St,40.754090,-74.031600,8 St & Washington St,40.745984,-74.028199,member


In [3]:
def convert_date(date):
    """
    Takes in date in either 
    
    "2020-11-01 00:53:21.2340"
    or 
    "2021-12-16 16:15:22" 
    
    format and returns a date in format "MM/DD/YYYY"
    
    If date is given in a different format np.nan is returned
    """
    if len(date) == 24:
        ndate = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S.%f')
        return ndate.strftime('%m/%d/%Y')

    if len(date) == 19:
        ndate = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        return ndate.strftime('%m/%d/%Y')
    else:
        return np.nan
    
def convert_time(date):
    """
    Takes in date in either 
    
    "2020-11-01 00:53:21.2340"
    or 
    "2021-12-16 16:15:22" 
    
    format and returns a time in format "HH:MM" (24 hr)
    
    If date is given in a different format np.nan is returned
    """
    if len(date) == 24:
        ndate = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S.%f')
        return ndate.strftime('%H:%M')

    if len(date) == 19:
        ndate = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        return ndate.strftime('%H:%M')
    else:
        return np.nan
    
def find_weekday(date):
    """
    Takes in date in either 
    
    "2020-11-01 00:53:21.2340"
    or 
    "2021-12-16 16:15:22" 
    
    format and returns a day of week
    
    If date is given in a different format np.nan is returned
    """
    if len(date) == 24:
        ndate = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S.%f')
        return calendar.day_name[ndate.weekday()]

    if len(date) == 19:
        ndate = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        return calendar.day_name[ndate.weekday()]
    else:
        return np.nan    
    
def impute_user(status):
    """
    Converts "Subscriber" to "member"
    
    AND
    
    Converts "Customer" to "casual"
    
    If neither "Suscriber" or "Customer" given, returns "unknown"
    """
    if status == "Subscriber" or status =="member":
        return "member"
    elif status == "Customer" or status =="casual":
        return "casual"
    else:
        return "unknown"
        
#Create new columns using defined functions
df["started_date"] = df["started_at"].map(lambda x: convert_date(str(x)))
df["started_time"] = df["started_at"].map(lambda x: convert_time(str(x)))
df["ended_date"] = df["ended_at"].map(lambda x: convert_date(str(x)))
df["ended_time"] = df["ended_at"].map(lambda x: convert_time(str(x)))
df["weekday"] = df["started_at"].map(lambda x: find_weekday(str(x)))
df["member_casual"] = df["member_casual"].map(lambda x: impute_user(x))
df

,started_at,ended_at,start_station_name,start_lat,start_lng,end_station_name,end_lat,end_lng,member_casual,started_date,started_time,ended_date,ended_time,weekday
0,2020-11-01 00:11:32.6250,2020-11-01 00:24:09.5850,Columbus Drive,40.718355,-74.038914,Baldwin at Montgomery,40.723659,-74.064194,member,11/01/2020,00:11,11/01/2020,00:24,Sunday
1,2020-11-01 00:14:20.4820,2020-11-01 00:24:04.7820,Grand St,40.715178,-74.037683,Brunswick St,40.724176,-74.050656,casual,11/01/2020,00:14,11/01/2020,00:24,Sunday
2,2020-11-01 00:22:55.8820,2020-11-01 00:29:39.0950,Hamilton Park,40.727596,-74.044247,Montgomery St,40.719420,-74.050990,member,11/01/2020,00:22,11/01/2020,00:29,Sunday
3,2020-11-01 00:41:37.4270,2020-11-01 00:49:55.7010,Lafayette Park,40.713464,-74.062859,Van Vorst Park,40.718489,-74.047727,casual,11/01/2020,00:41,11/01/2020,00:49,Sunday
4,2020-11-01 00:53:21.2340,2020-11-01 00:55:52.8060,City Hall,40.717732,-74.043845,Marin Light Rail,40.714584,-74.042817,member,11/01/2020,00:53,11/01/2020,00:55,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677320,2021-12-16 16:15:22,2021-12-16 16:32:43,Oakland Ave,40.737604,-74.052478,8 St & Washington St,40.745984,-74.028199,member,12/16/2021,16:15,12/16/2021,16:32,Thursday
677321,2021-12-27 23:57:45,2021-12-28 00:00:23,Clinton St & 7 St,40.745420,-74.033320,8 St & Washington St,40.745984,-74.028199,casual,12/27/2021,23:57,12/28/2021,00:00,Monday
677322,2021-12-21 13:32:14,2021-12-21 13:48:23,Clinton St & 7 St,40.745420,-74.033320,8 St & Washington St,40.745984,-74.028199,member,12/21/2021,13:32,12/21/2021,13:48,Tuesday
677323,2021-12-22 12:25:55,2021-12-22 12:34:17,Grand St & 14 St,40.754090,-74.031600,8 St & Washington St,40.745984,-74.028199,member,12/22/2021,12:25,12/22/2021,12:34,Wednesday


In [4]:
#Save to CSV
df.to_csv("citibike_14_months.csv")